# Steroscopic Image 

MPO file is a stereoscopic image that consists of two 2D .JPG images that are combined into one 3D image.
It may be created by digital cameras such as the Fujifilm FinePix Real 3D series or Nintendo 3DS.

The MPO format was developed by The Camera & Imaging Products Association (CIPA).
The specification actually allows for any number of JPG files to be saved in an MPO file,
but the most common use is for stereoscopic images.
Digital cameras that support the Multi Picture Object images include the Fujifilm FinePix Real 3D series cameras.

Standard JPG image viewers can also open MPO files, but you may only see one of the two images, generally the left image.

The files can also be converted to an anaglyph image and then viewed with red/green 3D glasses

Pillow identifies and reads Multi Picture Object (MPO) files, loading the primary image when first opened.
The seek() and tell() methods may be used to read other pictures from the file.
The pictures are zero-indexed and random access is supported.

index 0: left (red)
index 1: right (cyan)

use Pillow to process image and blend_modes to blend the image to anaglyph image

requires the PIL, numpy and blend_modes module 

In [ ]:
!pip install blend_modes

In [1]:
import os
from dataclasses import dataclass
import blend_modes  # type: ignore
import numpy as np
from PIL import Image

## image matrix:

* newRed   = 1*oldRed  +  0*oldGreen  +  0*oldBlue  + constant
* newGreen = 0*oldRed  +  1*OldGreen  +  0*OldBlue  + constant
* newBlue  = 0*oldRed  +  0*OldGreen  +  1*OldBlue  + constant

In [2]:
RED = (1, 0, 0, 0,
       0, 0, 0, 0,
       0, 0, 0, 0)
CYAN = (0, 0, 0, 0,
        0, 1, 0, 0,
        0, 0, 1, 0)

DEBUG = False


In [3]:
@dataclass 
class ImageFile:
    name: str
    left: str|None = None
    right: str|None = None
    blend: str|None = None 
        
    def __post_init__(self):
        path, base = os.path.split(self.name)
        base = '.'.join(base.split('.')[:-1])
        self.left = f'{path}/{base}-left.png'
        self.right = f'{path}/{base}-right.png'
        self.blend = f'{path}/{base}-blend.png'

In [4]:
def split_mpo(image_file:ImageFile)->None:
    """ split mpo into left.jg and right.jpg 
    """
    left_image = 0
    right_image = 1
    
    image = Image.open(image_file.name)
    if image.format != 'MPO':
        raise ValueError('not MPO image')
    try:
        image.seek(right_image)
    except EOFError:
        # pylint: disable=raise-missing-from
        raise EOFError('not seekable')
    image.save(image_file.right)
    image.seek(left_image)
    image.save(image_file.left)

In [5]:
def blend_image(image_file:ImageFile)->None:
    """ blend image 
    """
    left_image = Image.open(image_file.left)
    right_image = Image.open(image_file.right)
    
    left_red_image = left_image.convert(mode='RGB', matrix=RED)
    right_cyan_image = right_image.convert(mode='RGB', matrix=CYAN)
    
    image1 = left_red_image.convert(mode='RGBA')
    image2 = right_cyan_image.convert(mode='RGBA')
    image1_np = np.array(image1).astype(float)
    image2_np = np.array(image2).astype(float)

    blended_image_np = blend_modes.difference(  # type: ignore
        image1_np, image2_np, opacity=1)
    blended_image = Image.fromarray(np.uint8(blended_image_np))  # type:ignore
    blended_image.save(image_file.blend)

In [ ]:
image_file = ImageFile('Pics/001.mpo')
split_mpo(image_file)
blend_image(image_file)

img = Image.open(image_file.blend)
img.show()